In [1]:
import os  
import openai 
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path("raj.env"))
openai.api_key=os.getenv("OPENAI_API_KEY")

In [2]:
from typing import List
from pydantic import BaseModel, Field

# Pydantic Syntax

Pydantic is a 'data validation library' for Python
1. Works with python type annotations. But rather than static type checking, they are atively used at runtime for data validation and conversion. 
2. Provides built-in methods to serialize/deserialize models to/form JSON, dictionalries etc
3. langchain leverages Pydantic to create JSON Scheme describing function.

A nommal interface in Python is described below and later we look at how Pydantic works.

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [12]:
class Class(BaseModel):
    students: List[pUser]
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

# Pydantic to Create OpenAI function definition

First we are going to create a base model based on which the function will be created. Instead of writing the functions ourselves we can crearte a base model and convert it using convert_pydantic_to_openai_function. 

The typical idea is the following: 

class name_the function(basemodel):
    """ Description of the functions"   ##-- it is required
    str_name : str = Field(description = "         ")
    Field is used to describe the parameters. 
    
    Above all remember that all the descriptions are esentially prompts. So make sure to write a good one.

In [13]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [14]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [15]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [16]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [17]:
convert_pydantic_to_openai_function(WeatherSearch1)

KeyError: 'description'

In [18]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [19]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
model = ChatOpenAI()

In [22]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [23]:
model_with_function = model.bind(functions=[weather_function])

In [24]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

# Forcing it to use a function

We can force the model to use a function

In [25]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [26]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [27]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

# Using in a chain
We can use this model bound to function in a chain as we normally would

In [28]:
from langchain.prompts import ChatPromptTemplate

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [30]:
chain = prompt | model_with_function

In [31]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

# Using multiple functions
Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [32]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [33]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [34]:
model_with_functions = model.bind(functions=functions)
model_with_functions.invoke("what is the weather in sf?")



AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [35]:
model_with_functions.invoke("what are three songs by taylor swift?")


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "artist_name": "taylor swift",\n  "n": 3\n}', 'name': 'ArtistSearch'}})

In [36]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')

# Notes: 


In this notebook the goal is to use a base model using Pydantic and later convert it to a openai function and how to incorporate the function in the llm model.
1. Write the basemodel
2. Convert the class model to a openai function 
3. Use that function in the openai call and use it
4. Use multiple function and llm decide which function to choose in order to answer better. 
